In [9]:
#AHORRO TOTAL ACUMULADO
def calcular_ahorro_total(aporte_mensual, tasa_interes_anual, años_aporte):
    tasa_interes_mensual = tasa_interes_anual / 12
    periodos = años_aporte * 12
    ahorro_total = aporte_mensual * ((1 + tasa_interes_mensual) ** periodos - 1) / tasa_interes_mensual
    return ahorro_total

#RENTA VITALICIA MENSUAL
def calcular_renta_vitalicia(ahorro_total, tasa_interes_anual, años_esperanza_vida):
    tasa_interes_mensual = tasa_interes_anual / 12
    periodos = años_esperanza_vida * 12
    renta_mensual = ahorro_total * (tasa_interes_mensual / (1 - (1 + tasa_interes_mensual) ** -periodos))
    return renta_mensual

#PARAMETROS
salario_mensual = 7000000
aporte_porcentaje = 0.16
aporte_mensual = salario_mensual * aporte_porcentaje
tasa_interes_anual = 0.04
años_aporte = 32
años_esperanza_vida = 20


ahorro_total = calcular_ahorro_total(aporte_mensual, tasa_interes_anual, años_aporte)
print(f"Ahorro total al jubilarse: ${ahorro_total:,.2f}")


renta_mensual = calcular_renta_vitalicia(ahorro_total, tasa_interes_anual, años_esperanza_vida)
print(f"Renta vitalicia mensual: ${renta_mensual:,.2f}")



Ahorro total al jubilarse: $869,901,327.02
Renta vitalicia mensual: $5,271,430.93


In [16]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def TMensual(Tmort):
    """
    Interpola los valores de l(x) en pasos de 1/12 (mensual) y calcula q(x) y p(x).

    Parameters:
    Tmort (DataFrame): DataFrame que contiene las columnas 'x' y 'l(x)'.

    Returns:
    DataFrame: DataFrame con valores interpolados de x, l(x), q(x) y p(x).
    """

    # Aseguramos que Tmort tiene las columnas adecuadas
    if 'x' not in Tmort.columns or 'l(x)' not in Tmort.columns:
        raise ValueError("El DataFrame debe contener las columnas 'x' y 'l(x)'")

    # Obtener los valores de x y l(x)
    x_values = Tmort['x'].values
    lx_values = Tmort['l(x)'].values

    # Crear una función de interpolación
    interp_function = interp1d(x_values, lx_values, kind='linear', fill_value='extrapolate')

    # Crear los nuevos valores de x con un paso de 1/12, hasta 110 (incluyendo 110)
    new_x = np.arange(x_values[0], 110 + 1/12, 1/12)  # Incrementos mensuales hasta 110

    # Calcular los valores de l(x) interpolados
    new_lx = interp_function(new_x)

    # Inicializar los arrays para q(x) y p(x)
    qx_values = np.zeros(len(new_x) - 1)  # Inicializar el array para q(x)
    px_values = np.zeros(len(new_x) - 1)  # Inicializar el array para p(x)

    # Calcular q(x) y p(x) para cada nuevo valor de x
    for i in range(len(new_x)-1):
        if new_lx[i] > 0:  # Evitar división por cero
            qx_values[i] = 1 - (new_lx[i + 1] / new_lx[i])  # q(x) = 1 - l(x + 1/12) / l(x)
        else:
            qx_values[i] = 0  # Si l(x) es 0, q(x) también debe ser 0

        px_values[i] = 1 - qx_values[i]  # p(x) = 1 - q(x)


    # Crear un nuevo DataFrame con los valores interpolados y calculados
    interpolated_df = pd.DataFrame({
        'x': new_x,  # Incluir hasta 110
        'l(x)': new_lx,  # Excluir el último por la misma razón
        'q(x)': np.append(qx_values, 1),  # Añadir el valor de q(110)
        'p(x)': np.append(px_values, 0)  # Añadir el valor de p(110)
    })

    return interpolated_df

def tmort_filtered(df,edad):
    qx_vec=df[df['x']>=edad]
    return qx_vec

In [17]:
import pandas as pd

TM_hombres = pd.read_excel('TM_hombres.xlsx')

print(TM_hombres.head())

    x     l(x)   d(x)      q(x)  e0(x)
0  15  1000000  485.0  0.000485   64.8
1  16   999515  496.0  0.000496   63.9
2  17   999019  509.0  0.000509   62.9
3  18   998510  522.0  0.000523   61.9
4  19   997988  537.0  0.000538   60.9


In [20]:
 T_hombre=TMensual(TM_hombres)
 T_hombre

,x,l(x),q(x),p(x)
0,15.000000,1000000.000000,0.000040,0.999960
1,15.083333,999959.583333,0.000040,0.999960
2,15.166667,999919.166667,0.000040,0.999960
3,15.250000,999878.750000,0.000040,0.999960
4,15.333333,999838.333333,0.000040,0.999960
...,...,...,...,...
1136,109.666667,121.333333,0.083104,0.916896
1137,109.750000,111.250000,0.090637,0.909363
1138,109.833333,101.166667,0.099671,0.900329
1139,109.916667,91.083333,0.110704,0.889296


In [21]:
import pandas as pd

TM_mujeres = pd.read_excel('TM_mujeres.xlsx')

print(TM_mujeres.head())

    x     l(x)   d(x)      q(x)  e0(x)
0  15  1000000  272.0  0.000272   70.0
1  16   999728  278.0  0.000278   69.1
2  17   999450  285.0  0.000285   68.1
3  18   999165  293.0  0.000293   67.1
4  19   998872  302.0  0.000302   66.1


In [22]:
 T_mujer=TMensual(TM_mujeres)
 T_mujer

,x,l(x),q(x),p(x)
0,15.000000,1000000.000000,0.000023,0.999977
1,15.083333,999977.333333,0.000023,0.999977
2,15.166667,999954.666667,0.000023,0.999977
3,15.250000,999932.000000,0.000023,0.999977
4,15.333333,999909.333333,0.000023,0.999977
...,...,...,...,...
1136,109.666667,295.333333,0.083239,0.916761
1137,109.750000,270.750000,0.090797,0.909203
1138,109.833333,246.166667,0.099865,0.900135
1139,109.916667,221.583333,0.110944,0.889056


In [26]:
edad_jubilacion=62
T_hombre=tmort_filtered(T_hombre,edad_jubilacion)
print(T_hombre.head(13))
T_hombre['n']=T_hombre['x'].apply(lambda X:X -edad_jubilacion)*12
i = 0.04/12
T_hombre['Vn'] = np.ones(len(T_hombre))*(1/(1+i))
T_hombre['Vn'] = T_hombre.apply(lambda row: row['Vn'] ** row['n'],axis=1)
sum_vect = T_hombre['p(x)']*T_hombre['Vn']
suma = sum_vect.sum()
print(suma)
c = ahorro_total/suma

print(c)

             x      l(x)      q(x)      p(x)
564  62.000000  897019.0  0.000766  0.999234
565  62.083333  896331.5  0.000767  0.999233
566  62.166667  895644.0  0.000768  0.999232
567  62.250000  894956.5  0.000768  0.999232
568  62.333333  894269.0  0.000769  0.999231
569  62.416667  893581.5  0.000769  0.999231
570  62.500000  892894.0  0.000770  0.999230
571  62.583333  892206.5  0.000771  0.999229
572  62.666667  891519.0  0.000771  0.999229
573  62.750000  890831.5  0.000772  0.999228
574  62.833333  890144.0  0.000772  0.999228
575  62.916667  889456.5  0.000773  0.999227
576  63.000000  888769.0  0.000856  0.999144
254.5545084058316
3417347.948264875
